In [11]:
# Dependencies
import requests
from config import api_key
import pandas as pd
import json
import os
from sqlalchemy import create_engine
import numpy as np
from pprint import pprint 


url = f"http://www.omdbapi.com/?apikey={api_key}&t="
#url = "http://www.omdbapi.com/apikey={api_key}/?i="


In [47]:
#imdb_data = "Data/unique_indmID.csv"
#imdb_id = pd.read_csv(imdb_data,skiprows=[1], low_memory=False)

In [7]:
url = "http://www.omdbapi.com/?i="
api = f"&apikey={api_key}"

In [8]:
# Who was the director of the movie Aliens?
movie = requests.get(url + "Aliens").json()
print(f'The Director of Aliens was {movie["Director"]}.')

KeyError: 'Director'

In [ ]:
movies =[]

if x["Year"] in movies == "2022":
    movies.append(x)


In [21]:
movie = requests.get(url + "Gladiator").json()
movie["Year"]

'2000'

In [9]:
# What was the movie Gladiator rated?
movie = requests.get(url + "Gladiator").json()
print(f'The rating of Gladiator was {movie["Rated"]}.')

KeyError: 'Rated'

In [11]:
# What year was 50 First Dates released?
movie = requests.get(url + "50 First Dates").json()
print(f'The movie 50 First Dates was released in {movie["Year"]}.')

The movie 50 First Dates was released in 2004.


In [10]:
# Who wrote Moana?
movie = requests.get(url + "Moana").json()
print(f'Moana was written by {movie["Writer"]}.')

KeyError: 'Writer'

In [13]:
# What was the plot of the movie Sing?
movie = requests.get(url + "Sing").json()
print(f'The plot of Sing was: {movie["Plot"]}')

The plot of Sing was: In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists find that their lives will never be the same.


In [58]:
imdb_data = pd.read_csv("Resources/imdb_top_1000.csv")
imdb_data


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN
